In [ ]:
%matplotlib inline

In [ ]:
from LIMxCMBL.init import *

In [ ]:
from tqdm import trange

In [ ]:
a = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
b = np.reshape(a, (12))
c = np.reshape(b, (3, 4))

In [ ]:
a

In [ ]:
b

In [ ]:
c

In [ ]:
a==c

In [ ]:
print(chibs.shape, deltas.shape)

In [ ]:
for ell_idx in trange(len(ells)):
    ell_curr = ells[ell_idx]
    oup_fname = '/scratch/users/delon/LIMxCMBL/dkparp_integral/ell_%.3f.npy'%(ell_curr)
    _chibs, _deltas = np.meshgrid(chibs, deltas, indexing='ij')

    _chibs  = np.reshape(_chibs,  (len(chibs) * len(deltas)))
    _deltas = np.reshape(_deltas, (len(chibs) * len(deltas)))

    kperp2s = ell_curr**2 / (_chibs**2 * (1 - _deltas**2))

    from scipy.integrate import quad, quad_vec
    from scipy.interpolate import interp1d

    for i in range(len(chibs)):
        for j in range(len(deltas)):
            assert(np.abs(kperp2s[i*len(deltas)+j] - ell_curr**2 / (chibs[i]**2 * (1 - deltas[j]**2))) < 1e-8)

    def integrand(kparp):
        return 2/(2*np.pi) * np.cos(kparp * 2 * _chibs * _deltas) * ccl.linear_matter_power(cosmo, np.sqrt(kparp**2 + kperp2s), 1)

    res, err = quad_vec(integrand, 0, np.inf,
                      epsabs = 0.0, epsrel=1e-3, limit=1123456, workers=32)

    oup = np.reshape(res, (len(chibs), len(deltas)))

#     print(chibs.shape, deltas.shape)
#     print(oup.shape)

    np.save(oup_fname, oup)
#     print('saved to', oup_fname)